Street Easy Project Notebook WIP

Pedro Vincenty

In [ ]:
#import relevant libraries
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time 
from bs4 import BeautifulSoup
import json
import pandas as pd
import xlsxwriter
from datetime import datetime
import math
import undetected_chromedriver as uc


In [ ]:
#Input budget and error handling to capture correct format
while True:    
    try:
        budget = int(input("What\'s your budget in USD? (e.g. 2000, 2200.  Minimum budget is 500)"))
        if budget < 500:
            print("No apartment listing is that cheap :/  Enter a budget of atleast 500 USD")
        else:
            formatted_number = "${:,.0f}".format(budget) #<<Number is formatted to match drop down values in 'Maximum Price'>>
            break
    except ValueError:
        print("Please enter a valid input (e.g. 2000, 2200)")

In [ ]:
#Capture number of beds
while True:    
    try:
        beds = int(input("Indicate number of bedrooms (4 or less) or 'Studio' for your search."))
        if beds in range(1,5):
            break
        else:
            print("Please enter a valid number of bedrooms (4 or less) or Studio for your search.")
    #Bandaid way to capture studio
    except ValueError:
        beds = 'Studio'
        break

In [ ]:
#User inputs neighborhood
#if multiple seperate by comma
neighborhood = input('Choose a neighborhood')
neighborhood = neighborhood.title().split(', ')

In [ ]:
# Implementing the code below as 'bot detection' pop ups would interfere with my program.  Detection protocols still persist - 
# the program has to be restarted if instance occurs
# source for below code: https://www.zenrows.com/blog/selenium-avoid-bot-detection#how-anti-bots-work

# Create Chromeoptions instance 
options = webdriver.ChromeOptions() 
 
# # Adding argument to disable the AutomationControlled flag 
# options.add_argument("--disable-blink-features=AutomationControlled") 
 
# # Exclude the collection of enable-automation switches 
# options.add_experimental_option("excludeSwitches", ["enable-automation"]) 
 
# # Turn-off userAutomationExtension 
# options.add_experimental_option("useAutomationExtension", False) 
 
# Setting the driver path and requesting a page 
driver = webdriver.Chrome(options=options) 
 
# Changing the property of the navigator value for webdriver to undefined 
# driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})") 
 
link = "https://streeteasy.com/"

# Go to streeteasy.com
driver.get(link)

driver.maximize_window()


In [ ]:
# Define a function to click on xp path
def xpath_click(xp):
    time.sleep(1)
    driver.find_element(By.XPATH,xp).click()

In [ ]:
#Click search neighborhood bar
xp = "//button[@class='styled__Trigger-sc-17kdwb4-0 jYBvIf']"
xpath_click(xp)

In [ ]:
##This does it - to fix the issue of one worded neighborhoods click on the child of the ul items who's innertext matches the neighborhoodvalue

inputElement = driver.find_element(By.XPATH,"//input[@class='styled__Input-rx5fk3-1 dsKxbI styled__Combobox-sc-13cfbmj-0 eiptWM']")
try:
    for i in neighborhood:
        inputElement.send_keys(i)
        xpathNeighborhoodParent = "//ul[@id='location-search-listbox']"
        parentNeighborhood = driver.find_element(By.XPATH,xpathNeighborhoodParent)
        # Construct the XPath expression for the specific child based on text
        xpath_specific_child = f".//*[text()='{i}']"

        # Find the specific child element
        specific_child_element = parentNeighborhood.find_element(By.XPATH, xpath_specific_child)

        # Click the specific child element
        specific_child_element.click()
except NoSuchElementException:
    print("Please enter a valid borough or neighborhood.\nRestart the program and try 'upper east side', 'Tribeca', 'williamsburg'")


In [ ]:
# Hits 'Done' so desired neighborhoods are registered
xp = "//button[@class='styled__ButtonActionBase-sc-1y07t72-1 iXfPo styled__Apply-sc-46qus7-6 kSVIHP']"
xpath_click(xp)


In [ ]:
# Clicks on the max price dropdown to incorporate the user's budget
xp = "//input[@name='max']"
xpath_click(xp)

In [ ]:
# Get the drop down values of prices 
prices = []
xpath_budget = "//li[@class='Typography__Body-sc-100sue7-0 styled__Option-sc-11fgb4v-3 bifKRW fwaebO']"
elements = driver.find_elements(By.XPATH, xpath_budget)
for i in elements:
    prices.append(i.get_attribute('innerHTML'))
#Turns list into integers to create index in next block of code
integer_list = [int(''.join(filter(str.isdigit, s))) for s in prices]

In [ ]:
# if a user provides an input that's not included in the drop down to take the next highest value.  
closestIndex = next((x for x in integer_list if x >= budget), budget)
closestIndexFormatted = "${:,.0f}".format(closestIndex) #<<Number is formatted to match drop down values in 'Maximum Price'>>

In [ ]:
# Trick to find the element that contains the provided budget
# Then somewhere below filterout apartments slightly above budget
# ** Need to disable 'focusout' and 'blur' event listeners won't run otherwise.  Implement this within code.
# ^FALSE - just click on the target child element through calling the parent element

# xpathBudgetParent = "//ul[@class='styled__Datalist-sc-11fgb4v-2 edORTp']"
# parentBudget = driver.find_element(By.XPATH,xpathBudgetParent)

# Construct the XPath expression for the specific child based on text
xpathBudgetChildPath = f"//ul[@id='TextField-withTextarea-listbox']//li[contains(text(), '{closestIndexFormatted}')]"

# Find the specific child element
specificChildElement = driver.find_element(By.XPATH, xpathBudgetChildPath)

# Click the specific child element
specificChildElement.click()

In [ ]:
#Hit Search
# Won't hit search if going back and forth between vs code and chrome
xp = "//button[@class='styled__ButtonActionBase-sc-1y07t72-1 iXfPo styled__SearchButton-sc-1v106as-9 bNZexs']"
searchButton = driver.find_element(By.XPATH,xp)
# searchButton.click()
#childNeighborhood = parentNeighborhood.find_elements(By.XPATH, "./*")
# Construct the XPath expression for the specific child based on text
searchButtonChildPath = "//i[@class='styled__Icon-sc-14klawj-0 erxHVj fa fa-search styled__SearchIcon-sc-1v106as-10 izzUWB']"
# # Find the specific child element
specific_search_button = searchButton.find_element(By.XPATH, searchButtonChildPath)
specific_search_button.click()
# xp = "//button[@class='styled__ButtonActionBase-sc-1y07t72-1 iXfPo styled__SearchButton-sc-1v106as-9 bNZexs']"
# searchButton = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xp)))
# searchButton.click()

# Click the specific child element
#specific_search_button.click()


In [ ]:
# Hit Beds drop down
time.sleep(3)
xp = "//button[@aria-label='Beds / Baths']"
driver.find_element(By.XPATH,xp).click()

In [ ]:
# Hit number of beds per user's input
# Eureka - click on parent if element is intercepted.  Makes sense no?
# https://www.educba.com/xpath-parent/
xp = f"//input[@value='{beds}']"
test = driver.find_element(By.XPATH,xp)
test.find_element(By.XPATH,"./..").click()


In [ ]:
#Hit Done for beds
xp = "//button[@aria-label='Done']"
time.sleep(.5)
driver.find_element(By.XPATH,xp).click()

----------

Scrapping Part

In [ ]:
# Define page source and initialize beautiful soup 
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')
apartments_content = soup.find_all('div', class_="listingCardBottom--upperBlock")

In [ ]:
# Define function that clicks for next page a "number of pages - 1" number of times so there isn't an error thrown when the program searches to
# Doesn't click next when the button is non existent
# else clause clicks for the next page and checks if next page xpath exists and loops accordingly.
def stop_extra_click(i, page_num):
    if i == page_num-1:
        time.sleep(2)
        return True
    elif page_num ==1:
        time.sleep(2)
        return True
    else:
        try:
            time.sleep(2)
            driver.find_element(By.XPATH,xp).click()
            return False
        # Was getting errors that the element I was searching for wasn't available due to it being 'covered'
        # Thanks to ChatGPT I learned about the ActionChains package and the below syntax
        except ElementClickInterceptedException:
            time.sleep(2)
            action_chains = ActionChains(driver)
            action_chains.move_to_element(driver.find_element(By.XPATH,xp)).click().perform()
            return False
        except NoSuchElementException:
            time.sleep(2)
            print("Reached end of apartment listings")
            return True

In [ ]:
# Get the number of pages that resulted from the user's search.  This will define the number of loops the program will run
pages = soup.find_all('li', class_='page')
try:
    page_num = pages[-1].text.strip()
    page_num = int(page_num)
except IndexError:
    page_num = 1


In [ ]:
# xpath for 'click next' button
xp = "//li[@class ='next']"
# Lists to capture relevant data
addyList = []
prices = []
fees=[]
links = []
#Loop to cycle through all pages
for i in range(page_num):
    # Take content from each apartment
    for apartment in apartments_content:
        addy = apartment.find('address',{'class':'listingCard-addressLabel listingCard-upperShortLabel'}).text.strip() # <<Get address>>
        price = apartment.find('span',{'class':'price listingCard-priceMargin'}).text.strip() # <<< Get price >>>
        fee = apartment.find('span',{'class':'NoFeeBadge NoFeeBadge--SRPCard'}) # <<< Get fee instance>>>
        if apartment.find('span',{'class':'NoFeeBadge NoFeeBadge--SRPCard'}) is None: #<< is there's no fee an error is thrown, so use below logic to render 'FEE'
            fee = 'FEE'
        else:
            fee = fee.text.strip() # <<< will return 'No Fee' >>>
        link = apartment.find('a').attrs.get('href')# <<< Get the link to the apartment listing  >>>
        addyList.append(addy)
        prices.append(price)
        fees.append(fee)
        links.append(link)

    #logic to break out of loop 
    #time.sleep(1)   
    if stop_extra_click(i,page_num) == True:
        break

-------------

Data Framing

In [ ]:
#create out list that will be a list of dicionaries
listings = []
#for loop that creates list of dictionaries
for i in range(0,len(addyList)):
    data = {
    'addy': addyList[i],
    'price':prices[i],
    'fee':fees[i],
    'link':links[i],
    }
    listings.append(data)


In [ ]:
#Convert prices to numeric values so prices over budget are filtered out in the next block
for obj in listings:
    obj['price'] = int(obj['price'].replace('$', '').replace(',', ''))

In [ ]:
#In the event no listings are posted this block of code catches the KeyError
try:
    listings = pd.DataFrame(listings)
    listings = listings[listings['price']<= budget]
    listings['price'] = listings['price'].map('${:,}'.format)
    listings.set_index('price', inplace = True)
except KeyError:
    print('No listings are available per your preferences')


In [ ]:
excel_file = f'{datetime.now().strftime("%B %d %Y")} Listings.xlsx'

# Open the Excel file with XlsxWriter engine for further customization
with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
    listings.to_excel(writer, sheet_name='Sheet1')

    #get the xlsxwriter workbook and worksheet objects
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    #get the number of rows of the DataFrame
    num_rows=listings.shape[0]

    #hyperlink format
    hyperlink_format = workbook.add_format({'color': 'blue', 'underline': True})

    #add hyperlinks to the 'link' column
    for row_num in range(1, num_rows + 1):
        try:
            hyperlink_formula = f'=HYPERLINK("{listings.at[row_num, "link"]}", "{listings.at[row_num, "link"]}")'
            worksheet.write_formula(row_num, listings.columns.get_loc('link'), hyperlink_formula, hyperlink_format)
        except KeyError:
            pass

print(f"DataFrame with clickable links has been written to {excel_file}")

<a class="listingCard-globalLink jsGlobalListingCardLink" aria-labelledby="info-4289956" data-label-id="4289956-organic" se:behavior="hoverable selectable mappable" se:map="map" se:map:point="40.7678212,-73.95453409" data-map-points="40.7678212,-73.95453409" href="https://streeteasy.com/building/418-east-73-street-new_york/5fe" current-map-bounds="{&quot;northLat&quot;:40.81161383431982,&quot;southLat&quot;:40.74336863832152,&quot;eastLng&quot;:-73.89481525878908,&quot;westLng&quot;:-74.01978474121096}" current-map-center="{&quot;northSouthLat&quot;:40.7775,&quot;eastWestLng&quot;:-73.9573}"></a>